In [2]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline

In [3]:
#откроем данные
data = pd.read_csv('unconv.csv')
data.head()

,Well,Por,Perm,AI,Brittle,TOC,VR,Prod
0,1,12.08,2.92,2.80,81.40,1.16,2.31,4165.196191
1,2,12.38,3.53,3.22,46.17,0.89,1.88,3561.146205
2,3,14.02,2.59,4.01,72.80,0.89,2.72,4284.348574
3,4,17.67,6.75,2.63,39.81,1.08,1.88,5098.680869
4,5,17.52,4.57,3.18,10.94,1.51,1.90,3406.132832


## Задание 2.1

In [4]:
# Построим корреляционную матрицу факторов, включив в неё целевой признак.
corr_matrix = data.corr()
display(corr_matrix)
corr_matrix = np.array(corr_matrix)

,Well,Por,Perm,AI,Brittle,TOC,VR,Prod
Well,1.000000,0.068927,0.077928,0.041483,-0.079252,0.022624,-0.007279,0.026817
Por,0.068927,1.000000,0.760546,-0.461549,-0.218570,0.711831,0.111860,0.861910
Perm,0.077928,0.760546,1.000000,-0.239636,-0.124017,0.471746,0.051023,0.727426
AI,0.041483,-0.461549,-0.239636,1.000000,0.127599,-0.531864,0.499143,-0.390835
Brittle,-0.079252,-0.218570,-0.124017,0.127599,1.000000,-0.214282,0.317929,0.237155
TOC,0.022624,0.711831,0.471746,-0.531864,-0.214282,1.000000,0.299483,0.654445
VR,-0.007279,0.111860,0.051023,0.499143,0.317929,0.299483,1.000000,0.323182
Prod,0.026817,0.861910,0.727426,-0.390835,0.237155,0.654445,0.323182,1.000000


In [5]:
# ранг матрицы корреляций
np.linalg.matrix_rank(corr_matrix)

8

In [6]:
# определитель матрицы корреляций
np.linalg.det(corr_matrix)

0.0007299388072652095

## Задание 2.2

In [7]:
# матрица наблюдений X, вектор правильных ответов y
X = np.array(data.drop('Prod', axis=1))
y = np.array(data['Prod'])

In [8]:
# модель линейной регрессии по методу наименьших квадратов
# используя матричную формулу NumPy
n = X.shape[0]
X1 = np.column_stack((np.ones(n), X))
w_hat = np.linalg.pinv(X1.T@X1)@X1.T@np.array(y)
# полученные оценки коэффициентов модели
print(w_hat)
print(np.around(w_hat))

[-1.23230803e+03  5.07003631e-02  2.30179140e+02  1.16239006e+02
 -3.65202301e+02  2.49943700e+01 -7.84009294e+01  7.85259815e+02]
[-1232.     0.   230.   116.  -365.    25.   -78.   785.]


## Задание 2.3

In [9]:
# прогноз выработки газа для скважины с параметрами, указанными ниже
x_new = np.array([1, 106, 15.32, 3.71, 3.29, 55.99, 1.35, 2.42])
y_real = 4748.315024
y_predicted = w_hat@x_new
# абсолютная ошибка построенного прогноза для предложенной скважины (в миллионах кубических футов в день)
print(abs(y_predicted-y_real))

25.25097029324843


Метрика MAPE:

![](https://miro.medium.com/max/596/1*vmIM1jePXsDxPpgWEQ35Lg.png)

In [10]:
# прогноз выработки газа для всех скважин из обучающего набора данных
from sklearn.metrics import mean_absolute_percentage_error
y_pred = X1 @ w_hat
# значение метрики MAPE вашей модели
print('MAPE для модели линейной регрессии {:.2f}%'.format(mean_absolute_percentage_error(y, y_pred)*100))

MAPE для модели линейной регрессии 3.63%


## Задание 2.5

In [11]:
# Исключим из данных сильно коррелированные между собой факторы
data_drop = data.drop(['Well', 'Perm', 'TOC'], axis=1)
X2 = np.array(data_drop.drop('Prod', axis=1))
corr_matrix = data_drop.corr()
display(corr_matrix)

,Por,AI,Brittle,VR,Prod
Por,1.000000,-0.461549,-0.218570,0.111860,0.861910
AI,-0.461549,1.000000,0.127599,0.499143,-0.390835
Brittle,-0.218570,0.127599,1.000000,0.317929,0.237155
VR,0.111860,0.499143,0.317929,1.000000,0.323182
Prod,0.861910,-0.390835,0.237155,0.323182,1.000000


In [12]:
# Построим линейную регрессию на обновлённых после удаления факторов данных по методу наименьших квадратов
n = X2.shape[0]
X3 = np.column_stack((np.ones(n), X2))
w_hat2 = np.linalg.pinv(X3.T@X3)@X3.T@np.array(y)
# полученные оценки коэффициентов модели
print(w_hat2)
print(np.around(w_hat2))

[-1835.44646069   293.03624565  -200.03091206    27.64098209
   517.40272597]
[-1835.   293.  -200.    28.   517.]


In [13]:
# прогноз для всего обучающего набора данных
y_pred2 = X3 @ w_hat2
# метрика MAPE (Mean Absolute Percentage Error)
print('MAPE для модели линейной регрессии на сокращенном наборе факторов {:.2f}%'.format(mean_absolute_percentage_error(y, y_pred2)*100))

MAPE для модели линейной регрессии на сокращенном наборе факторов 4.04%


## Задание 3.1

In [14]:
# Сгенерируем полиномиальные признаки третьего порядка на выбанных факторах
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=3, include_bias=False)
X_poly = poly.fit_transform(X2)
#display(pd.DataFrame(X_poly))
print('Количество факторов стало', X_poly.shape[1])

Количество факторов стало 34


In [15]:
# модель линейной регрессии из библиотеки sklearn (LinearRegression) на полученных полиномиальных факторах
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
# вычисляем коэффициенты регрессии
lr.fit(X_poly, y)
#print('w_hat:', np.around(lr.coef_))

LinearRegression()

In [16]:
# среднее значение метрики MAPE, кросс-валидация на пяти фолдах
from sklearn.model_selection import cross_validate
cv_results = cross_validate(lr, X_poly, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.2f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.2f} %'.format(-cv_results['test_score'].mean() * 100))	

MAPE на тренировочных фолдах: 1.77 %
MAPE на валидационных фолдах: 2.68 %


## Задание 3.2

In [17]:
# стандартизируем данные
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() # инициализируем стандартизатор StandardScaler
X_st = scaler.fit_transform(X2) # подгоняем параметры стандартизатора
# добавляем полиномиальные признаки
poly = PolynomialFeatures(degree=3, include_bias=False)
X_poly_st = poly.fit_transform(X_st)
# модель Lasso на полиномиальных стандартизированных факторах. Коэффициент регуляризации = 5.
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=5)
lasso.fit(X_poly_st, y)
# среднее значение метрики MAPE, кросс-валидация на пяти фолдах
cv_results = cross_validate(lasso, X_poly_st, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.2f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.2f} %'.format(-cv_results['test_score'].mean() * 100))	

MAPE на тренировочных фолдах: 1.83 %
MAPE на валидационных фолдах: 2.28 %


## Задание 3.3

In [26]:
# стандартизируем данные
scaler = StandardScaler() # инициализируем стандартизатор StandardScaler
X_st = scaler.fit_transform(X2) # подгоняем параметры стандартизатора
# добавляем полиномиальные признаки
poly = PolynomialFeatures(degree=3, include_bias=False)
X_poly_st = poly.fit_transform(X_st)
# модель Ridge на полиномиальных стандартизированных факторах. Коэффициент регуляризации = 1.
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=1)
ridge.fit(X_poly_st, y)
#print(ridge.coef_)
# среднее значение метрики MAPE, кросс-валидация на пяти фолдах
cv_results = cross_validate(ridge, X_poly_st, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.2f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.2f} %'.format(-cv_results['test_score'].mean() * 100))	

MAPE на тренировочных фолдах: 1.83 %
MAPE на валидационных фолдах: 2.72 %


## Задание 3.4*

In [47]:
from sklearn.linear_model import ElasticNet
import warnings
warnings.filterwarnings('ignore')
# функция создания модели ElasticNet для заданного l1_ratio и расчета MAPE
def model_ElasticNet(l1, A, b):
    regr = ElasticNet(l1_ratio=l1)
    regr.fit(A, b)
    # среднее значение метрики MAPE, кросс-валидация на пяти фолдах
    cv_results = cross_validate(regr, A, b, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
    return (-cv_results['train_score'].mean()* 100, -cv_results['test_score'].mean() * 100)	

# стандартизируем данные
scaler = StandardScaler() # инициализируем стандартизатор StandardScaler
X_st = scaler.fit_transform(X2) # подгоняем параметры стандартизатора
# добавляем полиномиальные признаки
poly = PolynomialFeatures(degree=3, include_bias=False)
X_poly_st = poly.fit_transform(X_st)
# модель ElasticNet на полиномиальных стандартизированных факторах
# цикл, с измемнением l1_ratio
for l1_ratio in np.arange(0, 1.0001, 0.1):
    mape_train, mape_test = model_ElasticNet(l1_ratio, X_poly_st, y)
    print('При l1_ratio={:.2f} MAPE на тренировочных фолдах: {:.2f} %, MAPE на валидационных фолдах: {:.2f} %'.format(l1_ratio, mape_train, mape_test))


При l1_ratio=0.00 MAPE на тренировочных фолдах: 7.67 %, MAPE на валидационных фолдах: 8.88 %
При l1_ratio=0.10 MAPE на тренировочных фолдах: 7.45 %, MAPE на валидационных фолдах: 8.66 %
При l1_ratio=0.20 MAPE на тренировочных фолдах: 7.21 %, MAPE на валидационных фолдах: 8.41 %
При l1_ratio=0.30 MAPE на тренировочных фолдах: 6.93 %, MAPE на валидационных фолдах: 8.13 %
При l1_ratio=0.40 MAPE на тренировочных фолдах: 6.62 %, MAPE на валидационных фолдах: 7.81 %
При l1_ratio=0.50 MAPE на тренировочных фолдах: 6.25 %, MAPE на валидационных фолдах: 7.41 %
При l1_ratio=0.60 MAPE на тренировочных фолдах: 5.80 %, MAPE на валидационных фолдах: 6.93 %
При l1_ratio=0.70 MAPE на тренировочных фолдах: 5.24 %, MAPE на валидационных фолдах: 6.33 %
При l1_ratio=0.80 MAPE на тренировочных фолдах: 4.49 %, MAPE на валидационных фолдах: 5.56 %
При l1_ratio=0.90 MAPE на тренировочных фолдах: 3.40 %, MAPE на валидационных фолдах: 4.41 %
При l1_ratio=1.00 MAPE на тренировочных фолдах: 1.77 %, MAPE на валида

Минимальные значения MAPE при l1_ratio=1, то есть при L1 регуляризации. 

Однако получаем предупреждение:
 ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation